In [1]:
using Revise


In [15]:
push!(LOAD_PATH, "/uni-mainz.de/homes/ywitzky/phdscripts/Julia/Analysis/")
push!(LOAD_PATH, "/uni-mainz.de/homes/ywitzky/phdscripts/Julia/HPSAnalysis/")
push!(LOAD_PATH, "/uni-mainz.de/homes/ywitzky/phdscripts/Julia/HPSAnalysis/src/")
using ProteinSequences, Setup
using HPSAnalysis


In [3]:

NewPath = "/localscratch/HPS_DATA/HPS-Alpha/SLAB/HOOMD/"

Seitenlaenge =Dict("Test2"=>50.0,  "FUS"=>150.0,  "RS31a"=> 325.,"RS41"=> 325., "RS41_PHOS6"=> 325., "RS40"=> 325., "RS41_IDR2"=> 270.,"RS41_PHOS6_IDR2"=> 270., "RS41_RRM"=>270., "RS31a_IDR2"=> 170., "RS31"=>270);

In [4]:
using Printf


In [5]:
include("/uni-mainz.de/homes/ywitzky/phdscripts/Julia/HPSAnalysis/src/Setup/HOOMD_Setup.jl")


WriteParams (generic function with 1 method)

In [19]:
#pos = zeros(Float32,71400,3 )

if true
    local RunsPerProtein =ones(Int32, 100)
    local ToCreate = ["RS41_PHOS6"] #"RS31a_IDR2"["RS31a_IDR2", "RS41_RRM", "RS31", "RS40","RS31a", "RS41", "RS41_PHOS6", "RS41_IDR2", "RS41_PHOS6_IDR2"]  #"RS31",      "RS31a_IDR2", "RS41_RRM",         #["RS31", "RS40"] #IDR_Proteins#["RS41", "RS41_PHOS6"]
    local NChains=200 #200
    local Restart=false
    local ProteinNames = []
    local RelativePaths = []
    
    CoresPerRun=12
    ChargeSteps =[ 1 ]#,1.0, 1.2] ### must be even number of steps to be efficient

    for (protID, protein) in enumerate(ToCreate)
        for temp in [320] #Temperatures
            push!(ProteinNames, "$(protein)_slab")
            append!(RelativePaths, ["./$(protein)/$(temp)K"])

            pad = lpad(RunsPerProtein[protID],3,"0")
            mkpath(NewPath*"$(protein)/$(temp)K/")
            mkpath(NewPath*"$(protein)/$(temp)K/Restart/")
            mkpath(NewPath*"$(protein)/$(temp)K/HOOMD_Setup/")
            Path = NewPath*"$(protein)/$(temp)K/"
            #println(Path)
            cd(Path)
            SimName = "$(protein)_$(temp)_ChargeTemper"
            Info ="SLAB Simulation script for $protein.\n\n"
            BoxLengthShort=Float32(Seitenlaenge[protein])      #270 for IDRS, 325 for Non IDRS
            BoxLengthLong=Float32(1500.)
            BoxSize = [-BoxLengthShort/2., BoxLengthShort/2.,-BoxLengthLong/2., BoxLengthLong/2.,-BoxLengthShort/2., BoxLengthShort /2.]
            
            for (ChargeID, charge) in enumerate(ChargeSteps)
                Sequences= Vector{String}()
                Proteins = Vector{String}()
                for i in 1:NChains
                    push!(Proteins, protein)
                    push!(Sequences, ProteinSequences.NameToSeq[protein]) 
                end
                
                #=
                if Restart
                    if ~(temp in  keys(RestartRule))
                        println("Dont know how to restart!")
                        continue
                    end
                    local RunString = lpad(get!(RunNum, (protein, temp),1),3,"0")
                    local prot = HPSAnalysis.initData(NewPath*"$(protein)/$(RestartRule[temp])K/", "$(protein)_slab.lmp"; StepFrequency=1, Reparse=false, LoadAll=true);
                    #GSD.WriteAsPDB(prot,Start=100, Stop=prot.NSteps, Stepsize=100)
                    pos = HPSAnalysis.GetLastFrameForRestart(prot)
                    HPSAnalysis.SaveAllData(prot)
                else=#
                #=
                (pos, Data) = HPSAnalysis.CreateStartConfiguration(SimName,Path , [BoxLengthShort,BoxLengthShort , BoxLengthShort], Proteins, Sequences; Axis="y")
                    #pos = rand(sum(length.(Sequences)), 3)
                #end
                
                file = open("$(Path)/$(protein)_$(temp).xyz", "w");
                
                for atom in 1:size(pos,1)
                    write(file, "$(pos[atom, 1]) , $(pos[atom, 2]) , $(pos[atom, 3])\n")
                end
                close(file)
                =#
                
                pos = zeros(Float32,71400,3 )

                file = open("/localscratch/HPS_DATA/HPS-Alpha/SLAB/HOOMD/RS41_PHOS6/320K/RS41_PHOS6_320.xyz", "r");
                
                for atom in 1:size(pos,1)
                    #println(readline(file))
                    #println(split(readline(file)," , "))
                    pos[atom,:] = parse.(Float32, split(readline(file)," , "))
                    #write(file, "$(pos[atom, 1]) , $(pos[atom, 2]) , $(pos[atom, 3])\n")
                end
                close(file)
                

                #Setup.
                writeStartConfiguration("./$(protein)_slab","./$(SimName)_Start_slab_$(ChargeID)", Info, Sequences, BoxSize , 1_000_000_000; SimulationType="Calvados2+Alpha" , Temperature=temp,  InitStyle="Pos", Pos=pos , ChargeTemperSteps=ChargeSteps, ChargeTemperSwapSteps=10_000, HOOMD=true)
            end
        end
    end
    mkpath(NewPath*"SlurmScripts/")

end


[325.0, 1500.0, 325.0]
[138.91000366210938, 139.32998657226562, 141.26998901367188, 146.19000244140625, 151.27999877929688, 153.17001342773438, 156.20999145507812, -161.22999572753906, -157.3000030517578, -158.14999389648438, -160.92999267578125, -159.42999267578125, -157.3000030517578, -158.42999267578125, -160.0, 162.42999267578125, 158.54998779296875, 158.92001342773438, 159.05999755859375, 160.17999267578125, -161.13999938964844, -160.3000030517578, 160.95999145507812, -157.6999969482422, -153.24000549316406, -153.86000061035156, -158.3000030517578, -157.08999633789062, 160.739990234375, 162.42999267578125, 157.79998779296875]


Any[1, 2, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 11, 12, 13, 14, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 33, 34, 35, 35, 36, 37, 38, 39, 39, 40, 41, 42, 43, 44, 45, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 55, 56, 56, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 66, 67, 68, 69, 70, 71, 72, 73, 73, 74, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 96, 97, 98, 99, 100, 101, 102, 103, 104, 104, 105, 106, 107, 107, 108, 109, 110, 111, 112, 113, 114, 115, 115, 116, 117, 118, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 134, 135, 135, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 156, 157, 157, 158, 159, 159, 160, 161, 162, 163, 163, 164, 165, 166, 166, 167, 168, 168, 169, 170, 171, 172, 172, 173, 174, 175, 176, 177, 178, 179, 179, 180, 180, 181, 181, 182, 183, 184, 185, 186, 187, 188, 188, 189, 190, 191, 

"/localscratch/HPS_DATA/HPS-Alpha/SLAB/HOOMD/SlurmScripts"

In [7]:
pos = zeros(Float32,71400,3 )
file = open("/localscratch/HPS_DATA/HPS-Alpha/SLAB/HOOMD/RS41_PHOS6/320K/RS41_PHOS6_320.xyz", "r");
                
for atom in 1:size(pos,1)
    #println(readline(file))
    #println(split(readline(file)," , "))
    pos[atom,:] = parse.(Float32, split(readline(file)," , "))
    #write(file, "$(pos[atom, 1]) , $(pos[atom, 2]) , $(pos[atom, 3])\n")
end
close(file)

In [8]:
Sequences= Vector{String}()
Proteins = Vector{String}()
protein="RS41_PHOS6"
for i in 1:200
    push!(Proteins, "RS41_PHOS6")
    push!(Sequences, ProteinSequences.NameToSeq["RS41_PHOS6"]) 
end
BoxLengthShort=Float32(Seitenlaenge[protein])      #270 for IDRS, 325 for Non IDRS
BoxLengthLong=Float32(1500.)
AltBox = [BoxLengthShort,BoxLengthLong, BoxLengthShort ]

3-element Vector{Float32}:
  325.0
 1500.0
  325.0

In [9]:
altpos =reshape(pos, 200,357,3)
image = Setup.getImageCopyNumber(altpos, AltBox, Sequences);
println(image)

UndefVarError: UndefVarError: `Setup` not defined

In [18]:
using Dates

function writeStartConfiguration(fileName, StartFileName, Info, Sequences, BoxSize,NSteps=100000000; SimulationType="Calvados2", Temperature=300,MixingRule="1-1001-1", Pos =zeros(Float32, 0),InitStyle="Slab", SaltConcentration=-1, pH=6, ChargeTemperSteps=[], ChargeTemperSwapSteps=100_000, HOOMD=false)

    ChargeTemperSim=length(ChargeTemperSteps)>0

    NAtoms= 0
    NBonds=0
    NAngles=0
    NDihedrals=0
    for Seq in Sequences
        NAtoms += length(Seq)
        NBonds += length(Seq)-1
        NAngles += length(Seq)-2
        NDihedrals += length(Seq)-3
    end
    AlphaAddition=true
    if SimulationType=="Calvados2+Alpha"
        AlphaAddition = true
        SimulationType="Calvados2"
    else
        AlphaAddition=false
    end
  
    (AtomTypes, LongAtomTypes, AaToId, IdToAa,ResToLongAtomType, LongAtomTypesToRes, OneToCharge, OneToMass, OneToSigma, OneToLambda, OneToHPSDihedral0110, OneToHPSDihedral1001) =  Setup.DetermineCalvados2AtomTypes(Sequences, SimulationType, pH)
    NAtomTypes = length(LongAtomTypes)

    (dihedral_map, dihedral_eps) = Setup.determineDihedrals(Sequences, AtomTypes, AaToId, OneToHPSDihedral0110, OneToHPSDihedral1001, MixingRule)
    NDihedralsTypes = length(dihedral_eps)



    if InitStyle=="Slab"
        pos = createStartingPosition(Sequences, BoxSize)
    elseif (InitStyle=="Pos")
        cnt = 1
        pos_init = Pos

        pos = zeros(length(Sequences), maximum(length.(Sequences)), 3)
        for (Seqindex, Seq) in enumerate(Sequences)
            for (Aaid,_) in enumerate(Seq)
                pos[Seqindex, Aaid, 1] = pos_init[cnt,1]
                pos[Seqindex, Aaid, 2] = pos_init[cnt,2]
                pos[Seqindex, Aaid, 3] = pos_init[cnt,3]
                cnt+=1
            end
        end
    else
        pos = createDenseStartingPosition(Sequences, BoxSize)
    end

    AltBox = [BoxSize[2]-BoxSize[1], BoxSize[4]-BoxSize[3], BoxSize[6]-BoxSize[5]]
    println(AltBox)
    pos = Setup.correctPositionInBounds(pos, AltBox, Sequences) ### poly coordinates are not in box sometimes
    image = Setup.getImageCopyNumber(pos, AltBox, Sequences)
    #pos = Setup. rescalePositions(pos, AltBox, Sequences, image)
    println(pos[28,100:130,3])
    #function writeHPSLammpsScript(fileName, StartFileName, AtomTypes, Dihedral_eps, InitStyle, SimulationType, Restart, NTimeSteps)

    if HOOMD
        #println(pwd())
        WriteHOOMDSequences("./HOOMD_Setup/Sequences.txt", Sequences)
        WriteHOOMDParticlesInput("./HOOMD_Setup/Particles.txt", pos,  OneToCharge, AaToId,Sequences, OneToMass, OneToSigma, image)
        WriteDihedrals("./HOOMD_Setup/DihedralMap.txt", dihedral_map, dihedral_eps)

        BoxLength = [BoxSize[2]-BoxSize[1],BoxSize[4]-BoxSize[3],BoxSize[6]-BoxSize[5] ]
        WriteParams("./HOOMD_Setup/Params.txt", StartFileName, Temperature, NSteps, 100_000, 0.1, BoxLength/10.0, now().instant.periods.value%65535) ### BoxLength has to be convert to nm
        WriteDictionaries("./HOOMD_Setup/Dictionaries.txt", OneToCharge, AaToId, OneToMass, OneToSigma, OneToLambda)
    else
        writeHPSLammpsScript( fileName*".lmp",StartFileName*"$(ChargeTemperSim ?  "_\${id}" : "")", AtomTypes, LongAtomTypes, AaToId, LongAtomTypesToRes, OneToCharge, OneToSigma, OneToLambda, dihedral_eps, InitStyle, SimulationType, Temperature, AlphaAddition, false, NSteps; SaltConcentration=SaltConcentration, pH=pH, ChargeTemperSteps=ChargeTemperSteps, ChargeTemperSwapSteps=ChargeTemperSwapSteps)
        writeHPSLammpsScript( fileName*"_restart.lmp",StartFileName*"$(ChargeTemperSim ?  "_\${id}" : "")", AtomTypes, LongAtomTypes, AaToId, LongAtomTypesToRes, OneToCharge, OneToSigma, OneToLambda, dihedral_eps, InitStyle, SimulationType, Temperature, AlphaAddition, true, NSteps; SaltConcentration=SaltConcentration, pH=pH, ChargeTemperSteps=ChargeTemperSteps, ChargeTemperSwapSteps=ChargeTemperSwapSteps)

        file = open(StartFileName*".txt", "w");
        write(file, Info)

            write(file, "\n\t $NAtoms \t atoms\n")
            write(file, "\t $NBonds \t bonds\n")
            write(file, "\t $NAngles \t angles\n")
            write(file, "\t $NDihedrals \t dihedrals\n\n")

            write(file, "\t $NAtomTypes \t atom types\n")
            write(file, "\t 1 \t bond types\n")
            write(file, "\t 1 \t angle types\n")
            write(file, "\t $NDihedralsTypes \t dihedral types\n\n")

            write(file,"\t $(BoxSize[1]) \t $(BoxSize[2]) \t xlo xhi\n")
            write(file,"\t $(BoxSize[3]) \t $(BoxSize[4]) \t ylo yhi\n")
            write(file,"\t $(BoxSize[5]) \t $(BoxSize[6]) \t zlo zhi\n\n")

        write(file, "Masses \n#\n")
        for (index, value) in enumerate(LongAtomTypes)
            write(file, "\t $(AaToId[value]) \t $(OneToMass[value]) \t ### $(value in AtomTypes ? value : LongAtomTypesToRes[value] ) \n")
        end

        write(file,"\nAtoms\n# A comment that is needed to read stuff accurately\n")


        atomid=0
        moleculeid=0
        for (SeqId, Sequence) in enumerate(Sequences)
            moleculeid+=1
            for (ResId,Res) in enumerate(Sequence)
                atomid +=1
                write(file, "\t $atomid \t $(@sprintf("%6i", moleculeid)) \t  $(@sprintf("%2i", AaToId[Res])) \t  $(@sprintf("%3.5f", OneToCharge[Res])) \t $(@sprintf("%5.5f", pos[SeqId,ResId,1])) \t $(@sprintf("%5.5f", pos[SeqId,ResId,2])) \t $(@sprintf("%5.5f", pos[SeqId,ResId,3])) \t $(@sprintf("%i", image[SeqId,ResId,1])) \t $(@sprintf("%i", image[SeqId,ResId,2])) \t $(@sprintf("%i", image[SeqId,ResId,3]))\n")
            end
        end 

        write(file,"\nBonds\n#\n")
        atomid=0
        bondid=0
        for (SeqId, Sequence) in enumerate(Sequences)
            for (ResId,Res) in enumerate(Sequence)
                atomid +=1
                if ResId>(length(Sequence)-1)
                    continue
                else
                    bondid += 1
                    write(file, "\t $bondid \t 1 \t  $atomid \t  $(atomid+1) \n")
                end
            end
        # write(file, "\n")
        end 
        write(file, "\n\n")

        write(file,"\nAngles\n#\n")
        atomid=0
        angleid=0
        for (SeqId, Sequence) in enumerate(Sequences)
            for (ResId,Res) in enumerate(Sequence)
                atomid +=1
                if ResId>(length(Sequence)-2)
                    continue
                else
                    angleid += 1
                    write(file, "\t $angleid \t 1 \t  $atomid \t  $(atomid+1) \t $(atomid+2)\n")
                end
            end
        end 

        write(file, "\n\n")

        write(file,"\nDihedrals\n#\n")
        atomid=0
        dihedralid=0
        for (SeqId, Seq) in enumerate(Sequences)
            for (ResId,Res) in enumerate(Seq)
                atomid +=1
                if (ResId>(length(Seq)-3) )
                    continue
                else
                    dihedralid += 1
                    if MixingRule=="1-1001-1"
                        ID_min  = 
                        Res_min = (ResId-1)>=1 ? AaToId[Seq[ResId-1]] : 0
                        Res_max = (ResId)<=(length(Seq)-4) ? AaToId[Seq[ResId+4]] : -1
                        key = (sort([Res_min,AaToId[Res], AaToId[Seq[ResId+3]], Res_max]))
                    elseif MixingRule=="1001"
                        key = (sort([AaToId[Res], AaToId[Seq[ResId+3]]]))
                    elseif MixingRule=="0110"
                        key = (sort([AaToId[Seq[ResId+1]], AaToId[Seq[ResId+2]]]))
                    end
                    write(file, "\t $dihedralid \t $(dihedral_map[key]) \t  $atomid \t  $(atomid+1) \t $(atomid+2) \t $(atomid+3)\n")
                end
            end
        end 

        close(file)
    end
end


writeStartConfiguration (generic function with 2 methods)